In [ ]:
import numpy as np
import pandas as pd

In [ ]:
probs = pd.read_csv("probabilities.csv")
loss = pd.read_csv("t1_t5_loss.csv")

In [ ]:
print(probs.head())
print(loss.head())

In [ ]:
loss_1_yr = loss[loss["T_i"] == 2008]
loss_5_yr = loss[loss["T_i"] == 2012]
loss_1 = loss_1_yr[["LoanNum","Loss"]]
loss_5 = loss_5_yr[["LoanNum","Loss"]]
print(loss_1.head())
print(loss_5.head())

In [ ]:
probs = probs[["X1","X2","X3"]]

In [ ]:
actual = pd.read_excel("dataMoreFeatures.xlsx")
actual = actual[["LoanNum","GrossApproval"]]
print(actual.head())

In [ ]:
val_counts = probs.merge(actual,left_on = "X1",right_on = "LoanNum")
total_loan_amount = np.sum(val_counts["GrossApproval"])
print(total_loan_amount)

In [ ]:
new = probs.merge(loss_1,left_on = "X1",right_on = "LoanNum")
new2 = new.merge(loss_5,left_on = "X1",right_on = "LoanNum")
final = new2[["X2","X3","Loss_x","Loss_y"]]
print(final.head())

In [ ]:
num_simulations = 10000
total_loss = []
length = len(final)
for i in range(num_simulations):
    vec = np.random.uniform(low = 0, high = 1, size = (1,length))
    vec = np.array(vec).flatten()
    loss_vals = final[final["X3"] < vec]["Loss_x"]
    loss_vals = np.array(loss_vals)
    total_loss.append(loss_vals.sum())


In [ ]:
total_loss = np.array(total_loss)
print(len(total_loss))
print(np.mean(total_loss))

In [ ]:
### CDF
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
X1 = np.sort(total_loss)
Y1 = np.array(range(len(total_loss)))/float(len(total_loss))
plt.plot(X1,Y1)
plt.xlabel("Loss Amount")
plt.ylabel("Percentile")
#plt.savefig("CDF_of_Loss.png")
plt.show()

In [ ]:
### Histogram
n, bins, patches = plt.hist(x=total_loss, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.xlabel('Value')
plt.ylabel('Frequency')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
#plt.savefig("loss_histogram.png")

In [ ]:
### Histogram

q = total_loss/total_loan_amount
n, bins, patches = plt.hist(x=q, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.xlabel('Value')
plt.ylabel('Frequency')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
#plt.savefig("tranche_histogram.png")

In [ ]:
sorted_losses = np.sort(total_loss)
first_ind = int(0.95*10000)
second_ind = int(0.99*10000)
first_loss = sorted_losses[first_ind]
second_loss = sorted_losses[second_ind]
print("95% VAR: ", first_loss)
print("99% VAR: ", second_loss)

In [ ]:
### Average VAR
average_first_loss = np.mean(sorted_losses[first_ind:])
average_second_loss = np.mean(sorted_losses[second_ind:])
average_first_std = np.std(sorted_losses[first_ind:])
average_second_std = np.std(sorted_losses[second_ind:])
print("95% Average VAR", average_first_loss)
print("99% Average VAR", average_second_loss)

In [ ]:
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
estimate, lower, upper = mean_confidence_interval(sorted_losses[first_ind:])
estimate2, lower2, upper2 = mean_confidence_interval(sorted_losses[second_ind:])
print("Estimate, lower and upper for 95% AVAR", (estimate,lower,upper))
print("Estimate, lower and upper for 99% AVAR", (estimate2,lower2,upper2))